# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откроем файл с данными и изучим общую информацию

In [ ]:
import pandas as pd #вызов библиотеки pandas и сохранение как 'pd'
data = pd.read_csv('***') #чтение файла
data.info() #общая информация о данных
#оценила значения данных в таблице выводом на экран функции value_counts(), p.s.: закомментировала, чтобы программа долго не работала
print(data.head(10)) #вывели на экран 10 первых значений, чтобы ознакомиться с таблицей
for col in data.columns:
    print("-----------------------------")
    print(col)
    display(data[col].describe())
    display(data[col].value_counts())








**Вывод** : 

Из полученной информации можно сделать вывод, что всего у нас 21525 строк, в столбцах в количественных переменных days_employed и total_income есть пустые строки. Также делаем вывод о типе данных: столбцы children , dob_years ,education_id, family_status_id, debt - имеют целочисленные значения; days_employed, total_income - числа с плавающей точкой; остальные - текстовые или смешанные числовые и нечисловые значения.
Значения в столбце Дети имеют отрицательное значение, скорей всего это ошибка при записи, приведем потом значения столбца к модулю.
В столбце с возрастом есть '0', но этого не может быть, скорей всего просто не указали возраст при заполнении данных, поменяем нули на среднее значение.
Также можно заметить, что в данных трудового стажа есть отрицательные значения, это противоречит здравому смыслу. Скорей всего была допущена ошибка в добавлении '-'. Также есть аномально большие значения, скорей всего они записаны в часах, а не в днях.
П.С.: было бы неплохо уточнить у коллег, которые прислали данные, почему есть минусовые значения.

In [ ]:
display(data.head())

## Шаг 2. Предобработка данных

### Обработка пропусков

In [ ]:
# так как общий трудовой стаж в днях и ежемесяный доход - количественные переменные, то пропуски заменим медианным значением.
data['total_income'] = data['total_income'].fillna(data['total_income'].median()) #Заменили пропущенные значения в столбце total_income на медианное значение ежемесячного дохода

# приведем данные из столбца children к положительным значениям 
data['children'] = data['children'].apply(abs) #берем значения по модулю столбца Дети

# приведем данные из столбца days_employed к положительным значениям и поделим на 24ч
data['days_employed'] = data['days_employed'].apply(abs) #берем значения по модулю
data.loc[data['days_employed'] > 20000, 'days_employed'] = data.loc[data['days_employed'] > 20000, 'days_employed'] / 24 #получаем стаж в днях
data['days_employed'] = data['days_employed'].fillna(data['days_employed'].median()) #Заменили пропущенные значения в столбце days_employed на медианное значение трудового стажа

#Заменим нулевое значение возраста на медиану
median_dob_value = int(data['dob_years'].median())
data.loc[data['dob_years'] == 0, 'dob_years'] = median_dob_value
data.info() # проверим наличие пустых строк


        
    


**Вывод**

Провели обработку пропусков, заменили все пропущенный значения в столбцах 'Общий трудовой стаж' и 'ежемесячный доход' на медианные значения. Перед этим приведя данные в столбце 'Общий трудовой стаж' к положительным значениям и поработав с аномально большими значениями. Как видим из проверки, пропусков больше нет.

### Замена типа данных

In [ ]:
#Чтобы легче было работать с числами, заменим вещественные типы данных на целочисленные (это операция не сильно повлияет на итоговые значения анализа таблицы)
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')
data.info() #Проверим нашу таблицу


**Вывод**

Теперь вместо чисел с плавающей точкой у нас - целочисленные значения. С ними удобней работать и проще читать таблицу.

### Обработка дубликатов

In [ ]:
#Для начала приведем все символы в столбце education и gender к нижнему регистру:
data['education'] = data['education'].str.lower()
data['gender'] = data['gender'].str.lower()
print('Количество дубликатов до удаления', data.duplicated().sum()) # проверим количество дубликатов, если не большое количество, то их удаление будет не критично
data = data.drop_duplicates().reset_index(drop=True) # удалим дубликаты



In [ ]:
print('Количество дубликатов после удаления', data.duplicated().sum())

**Вывод**

После обработки данных все символы приведены к нижнему регистру и удалены дубликаты. Что можно подвердить крайней проверкой, где сумма дубликатов равно 0.

### Лемматизация

In [ ]:
#мы видим, что лемматизацию стоит провести для столбца purpose
unique_purpose = data['purpose'].unique() #получим уникальные значения столбца purpose
unique_purpose = ' '.join(unique_purpose) #склеим их в одну строку с помощью join
# для лемматизации данных импортируем библиотеку pymystem3:
from pymystem3 import Mystem
m = Mystem()
lemmas = m.lemmatize(unique_purpose)
#print(lemmas)
from collections import Counter
print(Counter(lemmas)) 
datas=['недвижимость', 'автомобиль', 'образование', 'жилье', 'операция', 'свадьба', 'строительство'] #выделили основные категории целей получения кредита

#создаем функцию, которая будет после лемитизации столбца purpose сравнивать и присваивать категории целей кредита 
def lemmas_func(row):
    lemmas = m.lemmatize(row)
    if (datas[0] in lemmas or datas[3] in lemmas):
        return 'недвижимость'
    if datas[1] in lemmas:
        return 'автомобиль'
    if datas[2] in lemmas:
        return 'образование'
    if datas[4] in lemmas:
        return 'операция'
    if datas[5] in lemmas:
        return 'свадьба'
    if datas[6] in lemmas:
        return 'строительство'
        
    return 'другое'
    

#Применяем функцию и создаем отдельный столбец purpose_group с соответствующими категориями     
data['purpose_group'] = data['purpose'].apply(lemmas_func)
print(data.head(10))

**Вывод**

Добавили отдельный столбец purpose_group, в котором указана классификацию клиентов по целям кредита.

### Категоризация данных

In [ ]:
#Проведем классификацию по наличию детей:
def children_func(row):
    if row == 0:
        return 'нет детей'    
    return 'есть дети'    

#Применяем функцию и создаем отдельный столбец having_children с соответствующими категориями     
data['having_children'] = data['children'].apply(children_func)

#Проведем классификацию по сумме ежемесячного дохода, где по средним данным доход ниже 35тыс - низкий,от 35 до 120тыс - средний, выше 120тыс - высокий:
def total_income_func(row):
    if row <= 35000:
        return 'низкий' 
    if row >120000:
        return 'высокий'
    return 'средний'    

#Применяем функцию и создаем отдельный столбец having_children с соответствующими категориями     
data['income_level'] = data['total_income'].apply(total_income_func)
print(data.head(10))



**Вывод**

Чтобы легче отвечать на заданные вопросы, мы классифицировали некоторые столбцы: цели кредита, наличие детей и ежемесячный  доход.

## Шаг 3. Ответим на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [ ]:
#Используем сводную таблицу с использованием pivot:
children_debt = data.pivot_table(index=['having_children'], values = 'debt', aggfunc = ['sum', 'count', 'mean'])
children_debt.columns = ['debt', 'total', '%']
children_debt['%'] = round(children_debt['%']*100,1)
#Получили таблицу, в строках которой указано наличие детей, а в столбцах: dept - количество клиентов с задолженностью, total - общее количество клиентов в категории и процент.
print(children_debt)



**Вывод**

Из таблицы можно сделать вывод, что количество клиентов с задолженностью и имеющих детей равен 9,2% от всех клиентов симеющих детей, а количесво клиентов с задолженностью и без детей равен 7,5% от всех клиентов без детей.
То есть клиенты без детей чаще возвращают кредит в срок.


In [ ]:
#Можно добавить также зависимость не только наличия детей, но и уровень дохода
children_debt = data.pivot_table(index=['having_children','income_level'], values = 'debt', aggfunc = ['sum', 'count', 'mean'])
children_debt.columns = ['debt', 'total', '%']
children_debt['%'] = round(children_debt['%']*100,1)

#Получили таблицу, в строках которой указано наличие детей, а в столбцах: dept - количество клиентов с задолженностью, total - общее количество клиентов в категории и процент.
print(children_debt)



**Вывод**:

Можно будет сделать вывод, что клиенты с детьми и с низким уровнем дохода вообще не задерживают выплаты и у них нет задолженностей.
А вот клиенты с детьми и средним заработком чаще остальных имеют проблемы с возвратом кредита в срок.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
#Используем сводную таблицу с использованием pivot:
family_status_debt = data.pivot_table(index=['family_status'], values = 'debt', aggfunc = ['sum', 'count', 'mean'])
family_status_debt.columns = ['debt', 'total', '%']
family_status_debt['%'] = round(family_status_debt['%']*100,1)
#Получили таблицу, в строках которой указано наличие детей, а в столбцах: dept - количество клиентов с задолженностью, total - общее количество клиентов в категории и процент.
print(family_status_debt)

**Вывод**

Из таблицы можно сделать вывод, что задолженность чаще возникает, если клиент не женат/не замужем или в гражданском браке.


- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [ ]:
# Используем сводную таблицу с использованием pivot:
income_level_debt = data.pivot_table(index=['income_level'], values = 'debt', aggfunc = ['sum', 'count', 'mean'])
income_level_debt.columns = ['debt', 'total', '%']
income_level_debt['%'] =  round(income_level_debt['%']*100,1)
#Получили таблицу, в строках которой указано наличие детей, а в столбцах: dept - количество клиентов с задолженностью, total - общее количество клиентов в категории и процент.
print(income_level_debt)


**Вывод**

Клиенты с низким доходом возвращают кредит в срок чаще,чем остальные. Но как видим, количество людей с низким доходом вообще не часто берут кредиты (всего 69 человек из 21454).

- Как разные цели кредита влияют на его возврат в срок?

In [ ]:
#Используем сводную таблицу с использованием pivot:
purpose_group_debt = data.pivot_table(index=['purpose_group'], values = 'debt', aggfunc = ['sum', 'count', 'mean'])
purpose_group_debt.columns = ['debt', 'total', '%']
purpose_group_debt['%'] = round(purpose_group_debt['%']*100,1)
#Получили таблицу, в строках которой указано наличие детей, а в столбцах: dept - количество клиентов с задолженностью, total - общее количество клиентов в категории и процент.
print(purpose_group_debt)

**Вывод**

Клиенты, берующие кредит на недвижимость возвращают кредит в срок чаще остальных. Любители новых кредитованных автомобилей делают это с меньшим удовольствием, у них больший процент зависимости возврата кредита в срок в отличае от других целевых категорий

## Шаг 4. Общий вывод

Посмотрев на таблицы можно сделать вывод, что на возврат кредита в срок влияют все факторы. Если уровень дохода низкий или нет детей, то возврат кредита в срок наиболее вероятен. Возвращают чаще кредиты в срок, которые оформляют на недвижимость. А семейное положение 'гражданский брак' или 'не женат/не замужем' усугубляет ситуацию возврата кредита.

Для более точного анализа можно дополнительно проверять несколько условий, например: влияет ли на выплаты кредита в срок не только наличие детей, но и уровень ежемесячного дохода. 
И, по-моему мнению, судя по данным процент не возврата кредита во всех ситуациях не такой уж и большой, менее 1%.